# Top-down face recognition with DeepFace

## Detect and crop faces to reduce noise for facial recognition

In [ ]:
# installs dedicated module for background removal https://github.com/Ir1d/image-background-remove-tool
%pip install torch pandas opencv-python carvekit --extra-index-url https://download.pytorch.org/whl/cpu

In [2]:
import cv2
import os
import pandas as pd
from PIL import Image

In [3]:
class InputImage:
    def __init__(self, img_name):
        self.img = cv2.imread(img_name)
        self.__name = img_name

    def __str__(self):
        return self.__name

In [ ]:
# download: https://github.com/opencv/opencv_zoo/blob/main/models/face_detection_yunet/face_detection_yunet_2023mar.onnx
detector = cv2.FaceDetectorYN.create("./utils/face_detection_yunet_2023mar.onnx",  "", (0, 0))

## Iterates over the images in a given folder, detects faces and performs a crop on the face

In [1]:
#Iterates over the files in a given folder to perform cropping
input_folder = '../../data/images/nytimes/'

In [ ]:
# Converts whatever is not a .jpg into a .jpg
def convert_img_format_to_jpg(image_path):
   with Image.open(image_path) as img:
      if img.format != 'JPEG':
         rgb_img = img.convert('RGB')
         jpg_path = os.path.splitext(image_path)[0] + '.jpg'
         rgb_img.save(jpg_path, 'JPEG')
         print(f"Converted {image_path} to {jpg_path}")
         return jpg_path
      return image_path

In [ ]:
def detect_and_crop_faces(image_path, detector, enlargement_factor=2):
   # Read input image
   loaded_img = InputImage(image_path)
   print(loaded_img)
   img = loaded_img.img

   # Check if the image was loaded correctly
   if img is None:
      print(f"Error: Unable to load image at {image_path}")
      return

   height, width, _ = img.shape
   detector.setInputSize((width, height))
   _, faces = detector.detect(img)

   # If faces exist
   if faces is not None:
      # Then crop
      for f, face in enumerate(faces):
         base_name = os.path.basename(loaded_img.__str__())
         name, ext = os.path.splitext(base_name)
         unique_face_filename = f"{input_folder}/detected-faces/{name}{ext}"
         # Available parameters: x1, y1, w, h, x_re, y_re, x_le, y_le, x_nt, y_nt, x_rcm, y_rcm, x_lcm, y_lcm
         (x, y, w, h) = face[:4]
         x = int(x)
         y = int(y)

         y1 = max(0, int(y - h * (enlargement_factor - 1) / 2))
         y2 = min(height, int(y + h * (1 + (enlargement_factor - 1) / 2)))
         x1 = max(0, int(x - w * (enlargement_factor - 1) / 2))
         x2 = min(width, int(x + w * (1 + (enlargement_factor - 1) / 2)))
         facecrop = img[y1:y2, x1:x2]

         cv2.imwrite(unique_face_filename, facecrop)
         convert_img_format_to_jpg(unique_face_filename)


In [ ]:
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path):
        detect_and_crop_faces(file_path, detector)

## Removes crops that are too small

In [ ]:
img_dir = f"{input_folder}/detected-faces/"

for filename in os.listdir(img_dir):
    filepath = os.path.join(img_dir, filename)
    with Image.open(filepath) as im:
        x, y = im.size
    totalsize = x*y
    if totalsize < 12100:
        os.remove(filepath)